In [1]:
from test import setup_macro, setup_sumo, setup_model

In [4]:
#surpress warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
import torch
import numpy as np
from hydra import initialize, compose

def test(config):

    with initialize(config_path="src/config"):
        cfg = compose(config_name="config", overrides= [f"{key}={value}" for key, value in config.items()])  # Load the configuration
        
    # Import simulator module based on the configuration
    simulator_name = cfg.simulator.name
    if simulator_name == "sumo":
        env, parser = setup_sumo(cfg)
    elif simulator_name == "macro":
        env, parser = setup_macro(cfg)
    else:
        raise ValueError(f"Unknown simulator: {simulator_name}")
    
    use_cuda = not cfg.model.no_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    model = setup_model(cfg, env, parser, device)

    print(f'Testing model {cfg.model.name} on {cfg.simulator.name} environment')
    episode_reward, episode_served_demand, episode_rebalancing_cost = model.test(10, env)

    print('Mean Episode Reward: ', np.mean(episode_reward), 'Std Episode Reward: ', np.std(episode_reward))
    print('Mean Episode Served Demand: ', np.mean(episode_served_demand), 'Std Episode Served Demand: ', np.std(episode_served_demand))
    print('Mean Episode Rebalancing Cost: ', np.mean(episode_rebalancing_cost), 'Std Episode Rebalancing Cost: ', np.std(episode_rebalancing_cost))

    ##TODO: ADD VISUALIZATION
    #return episode_reward, episode_served_demand, episode_rebalancing_cost


## Configuration Parameters

You can choose from the following options:

### Simulator:
- **macro**
- **sumo**

### Model:
- **sac** (Soft Actor-Critic)
- **a2c** (Advantage Actor-Critic)
- **no_rebalancing**
- **plus_one** 
- **equal_distribution** 



In [8]:
config = {
    "simulator.name": "macro",
    "model.name": "sac",
    "simulator.city": "nyc_brooklyn",
}
test(config)

Testing model sac on macro environment
Mean Episode Reward:  45719.85187578983 Std Episode Reward:  0.0
Mean Episode Served Demand:  2865.4 Std Episode Served Demand:  0.0
Mean Episode Rebalancing Cost:  445.8 Std Episode Rebalancing Cost:  0.0


In [6]:
config = {
    "simulator.name": "macro",
    "model.name": "equal_distribution",
    "simulator.city": "nyc_brooklyn",
}
test(config)

Testing model equal_distribution on macro environment
Mean Episode Reward:  49221.79495712959 Std Episode Reward:  1016.2832587489811
Mean Episode Served Demand:  2984.1 Std Episode Served Demand:  65.53235841933358
Mean Episode Rebalancing Cost:  522.8 Std Episode Rebalancing Cost:  59.27022861437266


In [7]:
config = {
    "simulator.name": "macro",
    "model.name": "equal_distribution",
    "simulator.city": "nyc_brooklyn",
    "model.cplexpath": None, 
}
test(config)

Testing model equal_distribution on macro environment
Mean Episode Reward:  49166.67480956457 Std Episode Reward:  718.4252287993113
Mean Episode Served Demand:  2981.6 Std Episode Served Demand:  54.380511214956414
Mean Episode Rebalancing Cost:  498.4 Std Episode Rebalancing Cost:  60.171754170873236
